In [15]:
# Import libraries and packages and modules

import pandas as pd
import datetime as dt
from datetime import datetime

#Pandas options and settings 

pd.set_option('display.max_columns',None)
pd.set_option('display.float_format',lambda x: '%.2f' % x)

# Read data from excel file and copy the dataframe

df_ = pd.read_excel(r"C:\Users\ExtraBT\Downloads\online_retail_II.xlsx", sheet_name="Year 2010-2011")
df=df_.copy()


 

In [2]:
#Descriptive statistics and understanding Online Retail dataset 

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom


In [3]:
#Is there any missing values  in the dataset?   

df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [4]:
#Unique number of products

 
df['StockCode'].nunique()  

4070

In [36]:
 df['Description'].value_counts().head()
 


WHITE HANGING HEART T-LIGHT HOLDER    2028
REGENCY CAKESTAND 3 TIER              1724
JUMBO BAG RED RETROSPOT               1618
ASSORTED COLOUR BIRD ORNAMENT         1408
PARTY BUNTING                         1397
Name: Description, dtype: int64

In [18]:
# most selling products in descending order

df.groupby('Description').agg(
    {'Quantity':'sum'}).sort_values(
    'Quantity',ascending=False).head()
 

 

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53847
JUMBO BAG RED RETROSPOT,47363
ASSORTED COLOUR BIRD ORNAMENT,36381
POPCORN HOLDER,36334
PACK OF 72 RETROSPOT CAKE CASES,36039


In [22]:
# Data Preparation
    
#Remove missing values

df.dropna(inplace=True)
df.isnull().values.any()
df.describe().T


,count,mean,std,min,25%,50%,75%,max
Quantity,397925.00,13.02,180.42,1.00,2.00,6.00,12.00,80995.00
Price,397925.00,3.12,22.10,0.00,1.25,1.95,3.75,8142.75
Customer ID,397925.00,15294.31,1713.17,12346.00,13969.00,15159.00,16795.00,18287.00
TotalPrice,397925.00,22.39,309.06,0.00,4.68,11.80,19.80,168469.60


In [20]:
# Remove cancelled orders

df=df[~ df["Invoice"].str.contains("C",na=False)]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom


In [21]:
#Create 'TotalPrice' variable  

df['TotalPrice']=df['Price']*df['Quantity']
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.00,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.00,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.00,United Kingdom,20.34


In [23]:
#Calculating RFM metrics (recency,frequency,monetary)

# recency: Time since last order  
#frequency: Total number of transactions
#monetary:Total transaction value


#Most recent invoice date in the dataset 

df['InvoiceDate'].max() #2011-12-09 12:50:00'

Timestamp('2011-12-09 12:50:00')

In [25]:
#Analysis date

today_date=dt.datetime(2011,12,11)
today_date

datetime.datetime(2011, 12, 11, 0, 0)

In [28]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days, #recency
                                     'Invoice': lambda Invoice: Invoice.nunique(), #frequency
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})  # monetary

rfm.columns = ['recency', 'frequency', 'monetary']

#Removing zero monetary values

rfm = rfm[rfm["monetary"] > 0]

rfm.reset_index(inplace=True)

rfm.head()

,Customer ID,recency,frequency,monetary
0,12346.00,326,1,77183.60
1,12347.00,3,7,4310.00
2,12348.00,76,4,1797.24
3,12349.00,19,1,1757.55
4,12350.00,311,1,334.40


In [29]:
#Calculating RFM scores and creating RFM_SCORE variable

rfm['recency_score']=pd.qcut(rfm['recency'],5,labels=[5,4,3,2,1])

rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

rfm['monetary_score ']=pd.qcut(rfm['monetary'],5,labels=[1,2,3,4,5])

rfm.head()

,Customer ID,recency,frequency,monetary,recency_score,frequency_score,monetary_score
0,12346.00,326,1,77183.60,1,1,5
1,12347.00,3,7,4310.00,5,5,5
2,12348.00,76,4,1797.24,2,4,4
3,12349.00,19,1,1757.55,4,1,4
4,12350.00,311,1,334.40,1,1,2


In [30]:
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))
rfm.head()

# Creating & Analysing RFM Segments

seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
rfm.head()

,Customer ID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
0,12346.00,326,1,77183.60,1,1,5,11,hibernating
1,12347.00,3,7,4310.00,5,5,5,55,champions
2,12348.00,76,4,1797.24,2,4,4,24,at_Risk
3,12349.00,19,1,1757.55,4,1,4,41,promising
4,12350.00,311,1,334.40,1,1,2,11,hibernating


In [31]:
rfm.groupby('segment').agg({'mean','median','count','sum'})

Customer ID                            recency          \
                         median         sum count     mean  median     sum   
segment                                                                      
about_to_sleep         14970.50  5276491.00   352 14990.03      53   18766   
at_Risk                15759.00  9304871.00   593 15691.18     139   91195   
cant_loose             15827.00   988601.00    63 15692.08     108    8377   
champions              15187.00  9694137.00   633 15314.59       5    4027   
hibernating            15042.00 16136317.00  1071 15066.59     219  233055   
loyal_customers        15756.00 12695003.00   819 15500.61      30   27525   
need_attention         15252.00  2882275.00   187 15413.24      52    9804   
new_customers          13965.50   586022.00    42 13952.90       8     312   
potential_loyalists    15495.50  7476594.00   484 15447.51      18    8421   
promising              14403.00  1332859.00    94 14179.35      23    2210   

                                 frequency                   monetary  \
                    count   mean    median   sum count  mean   median   
segment                                                                 
about_to_sleep        352  53.31         1   409   352  1.16   330.05   
at_Risk               593 153.79         3  1707   593  2.88   678.25   
cant_loose             63 132.97         7   528    63  8.38  2225.97   
champions             633   6.36         8  7860   633 12.42  2612.96   
hibernating          1071 217.61         1  1180  1071  1.10   296.25   
loyal_customers       819  33.61         5  5307   819  6.48  1740.48   
need_attention        187  52.43         2   435   187  2.33   638.06   
new_customers          42   7.43         1    42    42  1.00   274.50   
potential_loyalists   484  17.40         2   973   484  2.01   521.28   
promising              94  23.51         1    94    94  1.00   221.43   

                                              
                           sum count    mean  
segment                                       
about_to_sleep       166142.02   352  471.99  
at_Risk              643129.43   593 1084.54  
cant_loose           176157.82    63 2796.16  
champions           4341091.16   633 6857.96  
hibernating          523336.98  1071  488.64  
loyal_customers     2345818.94   819 2864.25  
need_attention       167856.41   187  897.63  
new_customers         16304.94    42  388.21  
potential_loyalists  503951.45   484 1041.22  
promising             27636.75    94  294.01

In [32]:
rfm['Customer ID'].nunique()  #4338 customers in total

4338

In [34]:
#Example
#Potential Loyalists segment

rfm[rfm["segment"] == "potential_loyalists"].head()

,Customer ID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
9,12356.00,23,3,2811.43,4,3,5,43,potential_loyalists
11,12358.00,2,2,1168.06,5,2,4,52,potential_loyalists
25,12375.00,11,2,457.50,5,2,2,52,potential_loyalists
32,12384.00,29,2,585.27,4,2,3,42,potential_loyalists
47,12406.00,23,2,3415.30,4,2,5,42,potential_loyalists


In [35]:
rfm[rfm["segment"] == "potential_loyalists"].index

Int64Index([   9,   11,   25,   32,   47,   65,   76,   87,   95,   98,
            ...
            4254, 4260, 4262, 4278, 4284, 4285, 4310, 4312, 4329, 4335],
           dtype='int64', length=484)